In [ ]:
# imports
import pandas as pd
import numpy as np
from scipy.stats import binom, ttest_rel

In [ ]:
# helpers
df_to_list  = lambda df, i: df.iloc[i][df.iloc[i].notnull()].tolist()                   # convert pandas row with nan to list without
remove_nan  = lambda df, i: list(map(lambda x: int(float(x)), df_to_list(df, i)))       # the pandas data frame is a parallogram with nan values as padding 
significant = lambda p_val, alpha: f"Significant: {'yes' if p_val < alpha else 'no'}\n" # finding significant?
round_p_val = lambda p_val: f"p-value:     {round(p_val, 5)}\n"                         # round p value
out_string  = lambda p, a, t: f"{significant(p, a)}{round_p_val(p)}Threshold:  {t}\n"   # format outstring
bootstrap   = lambda X: np.random.choice(len(X), len(X), replace=True)                  # sample with replacement

In [ ]:
days = 7
file = 'data_more.csv'

In [ ]:
df = pd.read_csv(file)              # read constructed data.csv file
df = df[df.columns[1:days + 7 + 2]][:150*days]   # limit to treatment and date columns
df = df[(df != 'forbidden').all(1)] # remove treatment samples that broke due to lacking upvote rights
df = df[df['treatment'] != -1]      # remove treatment samples that were deleted before we could upvote

In [ ]:
M = np.array([remove_nan(df, i) for i in range(df.shape[0])]) # convert pandas data frame to numpy array
T = (M[M[:, 0] == 1] - 2)[:, 1:]                              # treatment dataframe with our's and author's upvote REMOVED
C = (M[M[:, 0] == 0] - 1)[:, 1:]                              # control data frame with author's upvote removed

In [ ]:
threshs = [10 ** i for i in range(4)] # our hypothesis looks at the four different orders of magnitude
p_vals  = []                          # store p-values
n       = T.shape[0]                  # number of samples
alpha   = 0.05 / (len(threshs))       # significance level with bernferoni correction

In [ ]:
# for i in range(days):                       # for all the days after the first day (RESULT TRUE FOR ALL DAYS)
for thresh in threshs:                        # for the four upvote orderes of magnitude in focus
    r = np.sum(T[:,-1] > thresh)              # how big a fraction of samples are above the treshhold in Treatment
    p = np.sum(C[:,-1] > thresh) / C.shape[0] # what's the probability of being above the threshhold given no treatment?
    p_vals.append(1 - binom.cdf(r, n, p))     # calcaulte p-value (whether we are above thresh or not is a binary/binomial question)

In [ ]:
np.random.seed(42)
C_idx = np.random.choice(len(C), min([len(C), len(T)]), replace=False) 
T_idx = np.random.choice(len(T), min([len(C), len(T)]), replace=False) 
ttest_ind(T[T_idx, -1], C[C_idx, -1], alternative='greater')

In [ ]:
boot = np.sum(np.array(T_means) < np.array(C_means)) # count often bootstrap treament mean beats control
print('T below C:  ', boot / 10_000, end='\n\n')     # print that
for p_val, thresh in zip(p_vals, threshs):           # loop through threshs
    print(out_string(p_val, alpha, thresh))          # print nice text describing result